<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2024/blob/main/Seminars/S5_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оновные алгоритмы ML

### Семинар 5. Методы кросс-валидации (CV). StratifiedShuffleSplit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# Использование встроенных средств sklearn

Для начала сгенерируем искусственный набор данных, как в прошлом семинаре, просто запустим следующую ячейку

In [ ]:
X, Y = datasets.make_classification(n_features=2,
                                       n_redundant=0,
                                       n_informative=1,
                                       random_state=202,
                                       n_clusters_per_class=1,
                                       flip_y=0,
                                       n_samples=200)


Скопируйте и поменяйте код из лекции для Monte-Carlo cross-validation так, чтобы он использовал <a href='https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html'>StratifiedShuffleSplit</a>

Если функция написана правильно, должны получиться похожие (или такие же?) графики, как в лекции

In [ ]:
def plot_mccv_results(auroc, title):
    with plt.style.context(('ggplot')):
        plt.figure(figsize=(12, 6))
        plt.title(title, size=22)
        plt.bar(range(0, auroc.shape[0]), auroc, color='gray', alpha=0.7)
        plt.axhline(auroc.max(), color='k', linewidth=1, linestyle='--')
        plt.axhline(auroc.min(), color='k', linewidth=1, linestyle='--')
        plt.axhspan(auroc.min(), auroc.max(), alpha=0.2, color='steelblue')
        plt.ylim([0, auroc.max() + 0.1])
        plt.xlabel('Repetition')
        plt.ylabel('AUROC')
        plt.ylim([0.5, 1.0])
        plt.tight_layout()
        plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.3,
                                                    random_state=100,
                                                    stratify=Y)

clf = KNeighborsClassifier(n_neighbors=2, n_jobs=-1)

auroc_mccv_50 = get_monte_carlo_cv_scores(X_train, y_train, clf, 100, .5)
auroc_mccv_10 = get_monte_carlo_cv_scores(X_train, y_train, clf, 100, .1)
auroc_mccv_random = get_monte_carlo_cv_scores(X_train, y_train, clf, 100)

In [ ]:
plot_mccv_results(auroc_mccv_50, 'Monte-Carlo CV, test size 50%')

In [ ]:
plot_mccv_results(auroc_mccv_10, 'Monte-Carlo CV, test size 10%')

# "Парадокс"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

Создадим таблицу 200 на 10000 со "здоровыми" и "больными" пациентами и данными об "экспрессии" их генов.


In [ ]:
gene_count = 10000
genes = [f"Gene{ind}" for ind in range(gene_count)]
healthy = pd.DataFrame(np.random.normal(size=(100, gene_count),
                                        loc=0,
                                        scale=1),
                       columns=genes)
healthy['State'] = "H"
diseased = pd.DataFrame(np.random.normal(size=(100, gene_count),
                                         loc=0,
                                         scale=1),
                        columns=genes)
diseased['State'] = "D"
patients = pd.concat([healthy, diseased], axis=0)

In [ ]:
patients

In [ ]:
X = patients.drop("State", axis=1)
Y = patients['State']

In [ ]:
top_k = 10

diffs = X[Y == "H"].mean(axis=0) - X[Y == 'D'].mean(axis=0)
top = np.abs(diffs).sort_values(ascending=False)[0:top_k]
genes = top.index

In [ ]:
X_selected = X[genes]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, Y == "D", test_size=0.3)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
y_train_pred = model.predict_proba(X_train)[:, 1]
train_rocauc = roc_auc_score(y_score=y_train_pred, y_true=y_train)
train_prauc = average_precision_score(y_score=y_train_pred, y_true=y_train)
train_accuracy = accuracy_score(y_pred=y_train_pred > 0.5, y_true=y_train)
print("Train quality:")
print(f"ROCAUC : {train_rocauc:.02f}")
print(f"PRAUC : {train_prauc:.02f}")
print(f"Accuracy: accuracy {train_accuracy:.02f}")

In [ ]:
y_test_pred = model.predict_proba(X_test)[:, 1]
train_rocauc = roc_auc_score(y_score=y_test_pred, y_true=y_test)
train_prauc = average_precision_score(y_score=y_test_pred, y_true=y_test)
train_accuracy = accuracy_score(y_pred=y_test_pred > 0.5, y_true=y_test)
print("Test quality:")
print(f"ROCAUC : {train_rocauc:.02f}")
print(f"PRAUC : {train_prauc:.02f}")
print(f"Accuracy: accuracy {train_accuracy:.02f}")

Очевидно, что мы не могли выучить никаких разумных признаков. Однако, мы их все же выучили... Объясните полученный парадокс

Тем не менее, может мы просто что-то делаем не так. Попробуем встроенные в sklearn методы для выбора признаков

In [ ]:
from sklearn.feature_selection import (SelectKBest, SelectFdr, SelectFwe,
                                       SelectPercentile, f_classif)

In [ ]:
X_selected = SelectKBest(f_classif, k=10).fit_transform(X, Y)

Повторите все на отобранных стандартным методом sklearn признаках. Результат объясните

Что делает метод выделения признаков SelectFdr? Что такое FDR? Опишите ниже

Примените метод SelectFdr. Результат объясните

Почему вообще возникла проблема в начале? Объясните

Как бы эту проблему можно было бы избежать?